In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm

torch.manual_seed(1)

In [52]:
class Model(nn.Module):
    def __init__(self, tag_to_ix):
        super(Model, self).__init__()
        self.embedding_dim = 5
        self.hidden_dim = 4
        
        self.word_vocab_size = word_vocab_size
        self.word_emb_dim = word_emb_dim
        self.word_lstm_units = word_lstm_units
        self.char_vocab_size = char_vocab_size
        self.char_emb_dim = char_emb_dim
        self.char_lstm_units = char_lstm_units
        self.n_class = len(tag_to_ix) + 2  # +2 means <START> and <STOP> tags.
        
        # Character input
        self.char_embeddings = nn.Embedding(num_embeddings=char_vocab_size
                                            , embedding_dim=char_emb_dim
                                            , padding_idx=0
                                           )
        self.char_lstm = nn.LSTM(input_size=char_emb_dim
                                 , hidden_size=char_lstm_units
                                 , bidirectional=True
                                )
        
        # Word input
        self.word_embeddings = nn.Embedding(num_embeddings=word_vocab_size
                                            , embedding_dim=word_emb_dim
                                            , padding_idx=0
                                           )
        
        self.word_lstm = nn.LSTM(input_size=word_emb_dim + (char_emb_dim * 2)
                                 , hidden_size=word_lstm_units
                                 , bidirectional=True
                                )
        
        self.fc = nn.Linear(in_features=word_lstm_units, out_features=self.n_class)

        self.crf = CRF.CRF(tag_to_ix)

    def forward(self, x):
        lstm_feats = self._get_lstm_features(x)
        score, tag_seq = self.crf(lstm_feats)
        return lstm_feats, tag_seq
    
    def _get_lstm_features(self, chars, char_lengths):
        '''
        chars: (sentence_length, word_length) 
        char_lengths: (sentence_length)
        '''
        char_lengths, sorted_index = char_lengths.sort(0, descending=True)
        
        char_embs = self.char_embeddings(chars[sorted_index])
        packed = torch.nn.utils.rnn.pack_padded_sequence(char_embs, word_lengths, batch_first=True)
        
        l_packed_out, _ = self.char_lstm(packed)
        l_output, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(l_packed_out)
        l_output.transpose_(0, 1)
        
        l_char_embs = torch.Tensor(torch.zeros((output.size(0), output.size(2))))
        l_char_embs[sorted_index] = torch.Tensor([self._cat_lstm_last(char_feat, length) for word_feat, length in zip(l_output, char_lengths)])
        
        
    def _cat_lstm_last(self, output, length):
        return torch.cat((output[length - 1, :self.char_lstm_dim], output[0, self.char_lstm_dim:]))


In [47]:
x = torch.Tensor([[[1, 2, 3], [4, 5, 6], [4, 5, 7]], [[10, 20, 31], [14, 15, 61], [41, 15, 17]]])
print(x.shape)
x

torch.Size([2, 3, 3])


tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 4.,  5.,  7.]],

        [[10., 20., 31.],
         [14., 15., 61.],
         [41., 15., 17.]]])

In [50]:
a = torch.Tensor([1, 2, 3])
b = torch.Tensor([4, 5])
c = torch.Tensor([6])
torch.nn.utils.rnn.pack_padded_sequence([10, 10, 1])

TypeError: pack_padded_sequence() missing 1 required positional argument: 'lengths'